In [1]:
# Escoge 15 acciones del S&P 500 que esten listadas desde el 2015
# Simula con 5,000 portafolios
# Escoge el que tiene más ratio de sharpe
# Crea 3 modelos, CAPM, 3 factores y 5 factores
# Obten coeficientes y R2
# Explica los resultados 

In [2]:
!pip install pandas_datareader


Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas_datareader.data as web
import datetime
from sklearn.metrics import r2_score
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

# Definir las fechas de inicio y fin
start_date = datetime.datetime(2015, 1, 1)
end_date = datetime.datetime(2023, 12, 31)

# Descargar los factores Fama-French desde la web de Ken French
ff_factors = web.DataReader("F-F_Research_Data_5_Factors_2x3", "famafrench", start_date, end_date)

#ff_factores.head()

/tmp/ipykernel_18427/143104460.py:15: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factors = web.DataReader("F-F_Research_Data_5_Factors_2x3", "famafrench", start_date, end_date)
/tmp/ipykernel_18427/143104460.py:15: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factors = web.DataReader("F-F_Research_Data_5_Factors_2x3", "famafrench", start_date, end_date)


In [4]:
factors_df = ff_factors[0] / 100
factors_df.index = factors_df.index.to_timestamp()
factors_df.head()

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2015-01-01,-0.0311,-0.0092,-0.0358,0.0161,-0.0165,0.0
2015-02-01,0.0613,0.0032,-0.0186,-0.0112,-0.0182,0.0
2015-03-01,-0.0112,0.0307,-0.0037,0.0009,-0.0052,0.0
2015-04-01,0.0059,-0.0309,0.0182,0.0006,-0.0061,0.0
2015-05-01,0.0136,0.0083,-0.0114,-0.0179,-0.0074,0.0


In [5]:
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'JNJ', 'PG', 'JPM', 'V', 'XOM', 'KO', 'T', 'DIS', 'INTC', 'COST', 'HD']
St = yf.download(tickers, start_date, end_date, interval='1mo')['Adj Close']
rt = St.pct_change().dropna()

[*********************100%%**********************]  15 of 15 completed


In [6]:
df1 = pd.merge(rt, factors_df, on = 'Date') #Donde coincidan las fechas para que se junten
df1

,AAPL,AMZN,COST,DIS,GOOGL,HD,INTC,JNJ,JPM,KO,...,PG,T,V,XOM,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,,,,,,,,,,,,,,,
2015-02-01,0.096449,0.072293,0.027764,0.144239,0.046656,0.098927,0.006356,0.023667,0.134134,0.051737,...,0.017149,0.069627,0.064336,0.012812,0.0613,0.0032,-0.0186,-0.0112,-0.0182,0.0000
2015-03-01,-0.027549,-0.021202,0.067603,0.007783,-0.014095,-0.009935,-0.052803,-0.011763,-0.011423,-0.063510,...,-0.037472,-0.055266,-0.033889,-0.032756,-0.0112,0.0307,-0.0037,0.0009,-0.0052,0.0000
2015-04-01,0.005786,0.133513,-0.055775,0.036514,-0.010691,-0.053512,0.040934,-0.013916,0.044239,0.008546,...,-0.029656,0.060949,0.009784,0.027883,0.0059,-0.0309,0.0182,0.0006,-0.0061,0.0000
2015-05-01,0.040991,0.017663,-0.000489,0.015177,-0.006287,0.041503,0.058679,0.009475,0.046747,0.009862,...,-0.006156,0.016183,0.039818,-0.024837,0.0136,0.0083,-0.0114,-0.0179,-0.0074,0.0000
2015-06-01,-0.033206,0.011323,-0.052809,0.034158,-0.009682,-0.002603,-0.110854,-0.019665,0.030100,-0.042237,...,-0.001913,0.028373,-0.020566,-0.015397,-0.0153,0.0290,-0.0079,0.0044,-0.0158,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-01,0.044613,0.143480,0.016573,-0.141854,0.144681,-0.056864,0.012234,-0.052779,-0.010597,-0.069992,...,-0.083053,-0.097166,-0.050273,-0.136556,0.0035,-0.0038,-0.0772,-0.0181,-0.0722,0.0036
2023-06-01,0.095844,0.081108,0.054623,0.015007,-0.025800,0.103815,0.067969,0.075512,0.071697,0.009387,...,0.064842,0.013986,0.076519,0.058726,0.0646,0.0134,-0.0026,0.0218,-0.0162,0.0040
2023-07-01,0.012785,0.025468,0.041402,-0.004368,0.108772,0.074684,0.069677,0.012144,0.086084,0.036228,...,0.030051,-0.089655,0.001053,-0.000093,0.0321,0.0286,0.0411,-0.0057,0.0057,0.0045


In [7]:
def run_all_models(data, ticker):
    flag_data = data.copy()
    flag_data[ticker + '_minus_RF'] = data[ticker] - data['RF']
    y = flag_data[[ticker + '_minus_RF']] 

    # CAPM 
    X_capm = flag_data[['Mkt-RF']]
    model_capm = LinearRegression().fit(X_capm, y)
    prediction_capm = model_capm.predict(X_capm)
    r2_capm = r2_score(y_pred=prediction_capm, y_true=y)
    summary_capm = {str(X_capm.keys().values): list(model_capm.coef_[0])}

    # 3 factor model 
    X_3factor = flag_data[['Mkt-RF', 'SMB', 'HML']]
    model_3factor = LinearRegression().fit(X_3factor, y)
    prediction_3factor = model_3factor.predict(X_3factor)
    r2_3factor = r2_score(y_pred=prediction_3factor, y_true=y)
    summary_3factor = {str(X_3factor.keys().values): list(model_3factor.coef_[0])}
    
    
    # 5 factor
    X_5factor = flag_data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
    model_5factor = LinearRegression().fit(X_5factor, y)
    prediction_5factor = model_5factor.predict(X_5factor)
    r2_5factor = r2_score(y_pred=prediction_5factor, y_true=y)
    summary_5factor = {str(X_5factor.keys().values): list(model_5factor.coef_[0])}
    
    summary = {
        'CAPM': {'BETAS': summary_capm, 'R2': r2_capm},
        '3 factor': {'BETAS': summary_3factor, 'R2': r2_3factor},
        '5 factor': {'BETAS': summary_5factor, 'R2': r2_5factor}

    }
    return summary


In [8]:
sigma = rt.cov()
n_port = 5000
n_act = len(rt.keys())
rf = 0.1125

In [9]:
W = np.random.dirichlet((1,) * n_act, size = n_port)

In [10]:
valores_esperados_markowitz = W.dot(rt.mean()) * 252

volatilitys_markowitz = []
for i in W:
    variance = i.dot(sigma).dot(i.T)
    volatility = np.sqrt(variance) * np.sqrt(252)
    volatilitys_markowitz.append(volatility)

In [11]:
df = pd.DataFrame({
    'expected_value': valores_esperados_markowitz,
    'volatility': volatilitys_markowitz
})
df.head()

,expected_value,volatility
0,2.840864,0.673684
1,3.414031,0.743119
2,2.166902,0.827476
3,3.305847,0.666082
4,3.464952,0.687705


In [12]:
df['w_aapl'] = [i[0] for i in W]
df['w_msft'] = [i[1] for i in W]
df['w_amzn'] = [i[2] for i in W]
df['w_googl'] = [i[3] for i in W]
df['w_jnj'] = [i[4] for i in W]
df['w_pg'] = [i[5] for i in W]
df['w_jpm'] = [i[6] for i in W]
df['w_v'] = [i[7] for i in W]
df['w_xom'] = [i[8] for i in W]
df['w_ko'] = [i[9] for i in W]
df['w_t'] = [i[10] for i in W]
df['w_dis'] = [i[11] for i in W]
df['w_intc'] = [i[12] for i in W]
df['w_cost'] = [i[13] for i in W]
df['w_hd'] = [i[14] for i in W]

In [13]:
df['sharpe'] = (df.expected_value - rf) / df.volatility
df.head()

,expected_value,volatility,w_aapl,w_msft,w_amzn,w_googl,w_jnj,w_pg,w_jpm,w_v,w_xom,w_ko,w_t,w_dis,w_intc,w_cost,w_hd,sharpe
0,2.840864,0.673684,0.157801,0.048678,0.033311,0.130780,0.036322,0.000476,0.022928,0.227812,0.015269,0.109980,0.020421,0.116074,0.039116,0.013081,0.027951,4.049917
1,3.414031,0.743119,0.171402,0.002612,0.118332,0.070472,0.070184,0.014337,0.083974,0.010535,0.004254,0.075725,0.029666,0.023241,0.080180,0.206545,0.038542,4.442800
2,2.166902,0.827476,0.062949,0.055208,0.005727,0.275975,0.006859,0.002843,0.219202,0.024361,0.035216,0.000520,0.005050,0.009275,0.183945,0.053423,0.059448,2.482734
3,3.305847,0.666082,0.007717,0.054599,0.228144,0.062219,0.129585,0.100205,0.070692,0.036864,0.053768,0.026007,0.001875,0.206032,0.009087,0.006391,0.006815,4.794226
4,3.464952,0.687705,0.038428,0.230261,0.019047,0.004096,0.010951,0.092866,0.073162,0.165835,0.015608,0.152003,0.040242,0.111045,0.007076,0.030202,0.009176,4.874842


In [14]:
max_s = df[df.sharpe == df.sharpe.max()]
max_s

,expected_value,volatility,w_aapl,w_msft,w_amzn,w_googl,w_jnj,w_pg,w_jpm,w_v,w_xom,w_ko,w_t,w_dis,w_intc,w_cost,w_hd,sharpe
2225,4.823479,0.801989,0.038714,0.005458,0.063386,0.020399,0.061193,0.069086,0.055235,0.008356,0.039226,0.040244,0.524878,0.007201,0.010701,0.039981,0.015942,5.874122


In [15]:
df1['portafolio'] = (max_s.w_aapl * df['AAPL'] + 
                    max_s.w_msft * df['MSFT'] + 
                    max_s.w_amzn * df['AMZN'] +
                    max_s.w_googl * df['GOOGL'] + 
                    max_s.w_jnj * df['JNJ'] +
                    max_s.w_msft * df['MSFT'] + 
                    max_s.w_amzn * df['AMZN'] +
                    max_s.w_googl * df['GOOGL'] + 
                    max_s.w_jnj * df['JNJ'] + 
                    max_s.w_pg * df['PG'] +
                    max_s.w_jpm * df['JPM'] +
                    max_s.w_v * df['V'] +
                    max_s.w_xom* df['XOM'] +
                    max_s.w_ko * df['KO'] +
                    max_s.w_t * df['T'] +
                    max_s.w_dis * df['DIS'] +
                    max_s.w_intc * df['INTC'] +
                    max_s.w_cost * df['COST'] +
                    max_s.w_hd * df['HD'])

KeyError: 'AAPL'